In [11]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from rdkit import Chem
import seaborn as sns

import tmap, os

from molmap import dataset

from sklearn.model_selection import KFold

import matplotlib.pyplot as plt
%matplotlib inline
tqdm.pandas(ascii=True)
np.random.seed(123)



In [5]:
n_fold = 5

In [3]:
esol = dataset.load_ESOL(True)
lipop = dataset.load_Lipop(True)
malaria = dataset.load_malaria(True)
hiv = dataset.load_HIV(True)
bace = dataset.load_BACE(True)
bbbp = dataset.load_BBBP(True)

total samples: 1128
total samples: 4200
total samples: 9999


RDKit WARNING: [17:33:37] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [17:33:37] WARNING: not removing hydrogen atom without neighbors


total samples: 41127
total samples: 1513


RDKit ERROR: [17:33:39] Explicit valence for atom # 1 N, 4, is greater than permitted
RDKit WARNING: [17:33:39] WARNING: not removing hydrogen atom without neighbors
RDKit ERROR: [17:33:39] Explicit valence for atom # 6 N, 4, is greater than permitted
RDKit WARNING: [17:33:39] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [17:33:39] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [17:33:39] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [17:33:39] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [17:33:39] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [17:33:39] WARNING: not removing hydrogen atom without neighbors
RDKit ERROR: [17:33:39] Explicit valence for atom # 6 N, 4, is greater than permitted
RDKit WARNING: [17:33:39] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [17:33:39] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [17:33:

11 invalid smiles are removed
total samples: 2039


In [53]:
random_seeds = [8, 32, 64, 128, 512, 1024, 2048, 4096, 8192, 16384]
cols = ['rd_%s' % i for i in random_seeds]

data_save_folder = './rand_split_results/'
if not os.path.exists(data_save_folder):
    os.makedirs(data_save_folder)
    
    
for data in [esol, lipop, malaria, hiv, bace, bbbp ]:
    
    task_name = data.task_name
    df = pd.DataFrame(data.x, columns = ['smiles'])
    df[task_name] = data.y
        
    ncols = []
    for seed, col in zip(random_seeds, cols):
        kf = KFold(n_splits=5, shuffle = True, random_state=seed)
        gb = {}
        i = 1
        for tr, ts in kf.split(range(len(df))):
            gb.update(dict(zip(ts, [i for x in ts])))
            i += 1
        ncol = pd.Series(gb).sort_index()
        ncols.append(ncol)
        
    dfc = pd.concat(ncols, axis=1)
    dfc.columns = cols
    df.join(dfc).to_csv(os.path.join(data_save_folder, 'rd_split_%s.csv' % task_name))